In [1]:
import requests
import pandas as pd
import ast
from rapidfuzz import process, fuzz
import json
import csv
import numpy as np

platforms_df
images1

In [2]:
consoles_list=['Game Boy','Super Nintendo Entertainment System','Nintendo 64','PlayStation 5','Nintendo Switch 2','Xbox','PlayStation 2','Game Boy Color',
              'Nintendo Switch','Xbox Series X|S','Next Generation Xbox','Game Boy Advance','GameCube','Nintendo DS','Xbox 360','PlayStation 3','Wii','Nintendo 3DS', 
              'PlayStation 4'
             ]

In [3]:
g1=pd.read_csv('../data/games1_1.csv')

In [4]:
g2=pd.read_csv('../data/games2_1.csv')

In [5]:
g10=pd.read_csv('../data/games2_2.csv')

In [6]:
g11=pd.read_csv('../data/games3.csv')

In [7]:
g12=pd.read_csv('../data/games4.csv')

In [8]:
g13=pd.read_csv('../data/games2_3.csv')

In [9]:
g14=pd.read_csv('../data/games2_4.csv')

In [10]:
g15=pd.read_csv('../data/games5.csv')

In [11]:
g16=pd.read_csv('../data/games6.csv')

In [12]:
g17=pd.read_csv('../data/games2_5.csv')

In [13]:
g18=pd.read_csv('../data/games2_6.csv')

In [14]:
g3=pd.concat([g1,g2,g10,g11,g12,g13,g14,g15,g16,g17,g18],axis=0,ignore_index=True)

In [15]:
cols = ['characters', 'image','concepts','developers','first_appearance_characters','first_appearance_concepts',
        'first_appearance_locations','first_appearance_objects','franchises','genres','locations','publishers','themes']

for col in cols:
    def extract_names(x):
        # Convert string to list if needed
        if isinstance(x, str):
            try:
                x = ast.literal_eval(x)
            except:
                return ''
        # If it's a list of dicts, extract 'name'
        if isinstance(x, list):
            names = [d.get('name','') for d in x if isinstance(d, dict)]
            return ', '.join(names)
        return ''
    
    g3[col + '_name'] = g3[col].apply(extract_names)

In [16]:
g3.to_csv('all_giant_games.csv')

In [17]:
game_reviews=pd.read_csv('../data/reviews_and_sales.csv')

In [19]:
len(g3)

13542

In [18]:
g3['name'] = g3['name'].astype(str)
game_reviews['Name'] = game_reviews['Name'].astype(str)

# Compute similarity matrix in parallel
scores = process.cdist(
    game_reviews['Name'].tolist(),
    g3['name'].tolist(),
    scorer=fuzz.token_sort_ratio,
    workers=-1  # use all CPU cores
)

# Get best matches and their scores
best_match_idx = np.argmax(scores, axis=1)
best_match_score = scores[np.arange(scores.shape[0]), best_match_idx]
best_matches = [g3['name'].iloc[i] for i in best_match_idx]

# Add results back to reviews_and_sales
game_reviews['matched_name'] = best_matches
game_reviews['score'] = best_match_score

# Replace Name with matched_name if score >= threshold
threshold = 85
game_reviews.loc[game_reviews['score'] >= threshold, 'Name'] = \
   game_reviews.loc[game_reviews['score'] >= threshold, 'matched_name']

# Merge on aligned names
merged_df = pd.merge(
    g3,
    game_reviews,
    left_on='name',
    right_on='Name',
    how='inner'
)

# merged_df now contains both datasets combined
merged_df


,Unnamed: 0,deck,id,image,name,number_of_user_reviews,original_release_date,platforms,characters,concepts,...,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,matched_name,score
0,5,A spin-off of the NBA Jam arcade basketball se...,125,{'icon_url': 'https://www.giantbomb.com/a/uplo...,College Slam,0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.01,0.19,NaN,NaN,NaN,NaN,NaN,NaN,College Slam,100.000000
1,7,The one-on-one arcade fighting game that kicke...,243,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Street Fighter II: The World Warrior,0,1991-02-06,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.12,6.30,NaN,NaN,NaN,NaN,NaN,NaN,Street Fighter II: The World Warrior,100.000000
2,10,A side-scrolling platform game released in 199...,397,{'icon_url': 'https://www.giantbomb.com/a/uplo...,James Pond 2: Codename: RoboCod,0,1991-08-18,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.00,0.03,NaN,NaN,NaN,NaN,NaN,NaN,James Pond 2: Codename: RoboCod,88.135590
3,13,A European-focused spin-off of PGA Tour Golf I...,697,{'icon_url': 'https://www.giantbomb.com/a/uplo...,PGA European Tour,0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN,PGA European Tour,100.000000
4,14,The arcade sequel to the original MSX Parodius...,810,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Parodius,0,1990-04-25,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.02,0.33,NaN,NaN,NaN,NaN,NaN,NaN,Parodius,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8953,2201,Kao returns.,84756,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Kao the Kangaroo,0,2022-05-27,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.00,0.01,52.0,8.0,tbd,NaN,Titus Software,E,Kao the Kangaroo,100.000000
8954,2221,An upgraded version of 2013's JoJo's Bizarre A...,85666,{'icon_url': 'https://www.giantbomb.com/a/uplo...,JoJo's Bizarre Adventure: All Star Battle R,0,2022-09-01,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.01,0.58,71.0,53.0,8.3,55.0,CyberConnect2,T,JoJo's Bizarre Adventure: All Star Battle R,97.619049
8955,2221,An upgraded version of 2013's JoJo's Bizarre A...,85666,{'icon_url': 'https://www.giantbomb.com/a/uplo...,JoJo's Bizarre Adventure: All Star Battle R,0,2022-09-01,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.00,0.06,NaN,NaN,NaN,NaN,NaN,NaN,JoJo's Bizarre Adventure: All Star Battle R,97.619049
8956,2297,A adventure puzzle game where players are sent...,86215,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Escape Academy,0,2022-07-13,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.05,0.33,51.0,48.0,5.8,4.0,Shift,E10+,Escape Academy,87.500000


In [20]:
g4=merged_df

In [9]:
g4["platforms"] = g4["platforms"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [10]:
# Step 1: extract just the names of platforms per row
g4["platform_names"] = g4["platforms"].apply(
    lambda lst: [d.get("name") for d in lst] if isinstance(lst, list) else []
)

# Step 2: build indicator columns for each console in your list
console_games_df = pd.DataFrame(index=g4.index)

for console in consoles_list:
    console_games_df[console] = g4["platform_names"].apply(
        lambda names: console if console in names else "-"
    )

# Done: console_games_df now has one row per game, columns = consoles
console_games_df

,Game Boy,Super Nintendo Entertainment System,Nintendo 64,PlayStation 5,Nintendo Switch 2,Xbox,PlayStation 2,Game Boy Color,Nintendo Switch,Xbox Series X|S,Next Generation Xbox,Game Boy Advance,GameCube,Nintendo DS,Xbox 360,PlayStation 3,Wii,Nintendo 3DS,PlayStation 4
0,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,Game Boy,Super Nintendo Entertainment System,-,-,-,-,PlayStation 2,-,Nintendo Switch,-,-,Game Boy Advance,-,Nintendo DS,-,-,-,-,-
3,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4
1777,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4
1778,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4
1779,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4


In [11]:
g5=pd.concat([g4,console_games_df],axis=1)

In [21]:
cols = ['characters', 'image','concepts','developers','first_appearance_characters','first_appearance_concepts',
        'first_appearance_locations','first_appearance_objects','franchises','genres','locations','publishers','themes']

for col in cols:
    def extract_names(x):
        # Convert string to list if needed
        if isinstance(x, str):
            try:
                x = ast.literal_eval(x)
            except:
                return ''
        # If it's a list of dicts, extract 'name'
        if isinstance(x, list):
            names = [d.get('name','') for d in x if isinstance(d, dict)]
            return ', '.join(names)
        return ''
    
    g4[col + '_name'] = g4[col].apply(extract_names)

In [25]:
g4.to_csv('all_games_and_reviews.csv')

In [16]:
g5['image_name']

0        
1        
2        
3        
4        
       ..
1776     
1777     
1778     
1779     
1780     
Name: image_name, Length: 1781, dtype: object

In [27]:
len(g4)

8958